### Bibliotecas

In [ ]:
import pandas as pd
import os
import pickle
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

### Modelos para extraer embeddings

In [2]:
MODELOS = {
    "sentence-transformers": 'Santp98/SBERT-pairs-bert-base-spanish-wwm-cased',
    "sentence_similarity": 'hiiamsid/sentence_similarity_spanish_es'
}

CAMPOS = ["Titulo", "resumen", "objetivos", "claves"]

### Extracción de NER's

In [ ]:
def cargar_modelo_ner(nombre_modelo="iEsmeralda/mrm8488-finetuned-ner-tech"):
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo) 
    modelo = AutoModelForTokenClassification.from_pretrained(nombre_modelo)
    ner_pipeline = pipeline("ner", model=modelo, tokenizer=tokenizador, aggregation_strategy="simple")
    # aggregation_strategy="simple" indica que se deben agrupar los tokens que pertenezcan a una misma entidad, por ejemplo si el modelo detecta "inteligencia" y "artificial" 
    # como parte de una misma entidad, los junta en una sola: "inteligencia artificial"

    return ner_pipeline

# funcion para agrupar entidades consecutivas del mismo tipo, esto se hizo porque habia entidades que no fueron "unidas" a pesar del aggregation_strategy
def agrupar_entidades_consecutivas(entidades):
    # si no hay entidades, se regresa una lista vacia
    if not entidades:
        return []
    entidades_agrupadas = [] 
    entidad_actual = entidades[0].copy() # se toma la primera entidad como "base" 
    for i in range(1, len(entidades)):
        entidad = entidades[i]

        # si la entidad es del mismo tipo y esta justo despues de la actual, entonces:
        if entidad["entity_group"] == entidad_actual["entity_group"] and entidad["start"] == entidad_actual["end"] + 1:
            # se une la palabra, se actualiza el fin y se promedia el puntaje de ambas entidades
            entidad_actual["word"] += " " + entidad["word"]
            entidad_actual["end"] = entidad["end"]
            entidad_actual["score"] = (entidad_actual["score"] + entidad["score"]) / 2
        else:
            # si no son consecutivas, se guarda la actual y se pasa a la siguiente entidad 
            entidades_agrupadas.append(entidad_actual)
            entidad_actual = entidad.copy()
    entidades_agrupadas.append(entidad_actual)
    return entidades_agrupadas

# funcion para extraer las entidades nombradas de un texto largo
def extraer_ners(texto, ner_pipeline, max_tokens=512):
    tokenizador = ner_pipeline.tokenizer
    modelo = ner_pipeline.model

    tokens = tokenizador.tokenize(texto)
    entidades = []

    # se recorre la lista completa de tokens en bloques de tamaño max_tokens ya que la mayoria de los modelos de transformers solo aceptan secuencias de hasta 512 tokens como maximo
    for i in range(0, len(tokens), max_tokens):
        bloque_de_tokens = tokens[i:i+max_tokens]

        # la lista de tokens nombrada como bloque_de_tokens se pasa a texto plano porque el pipeline de ner espera un texto como entrada
        bloque_texto = tokenizador.convert_tokens_to_string(bloque_de_tokens)

        # se vuelve a tokenizar el texto del bloque para asegurarse que no se pase del limite real de tokens del modelo
        if len(tokenizador(bloque_texto)["input_ids"]) > max_tokens:
            # si el bloque excede el limite de 512 tokens, se salta 
            continue

        # se aplica el pipeline de ner al bloque de texto para obtener las entidades detectadas en ese bloque
        entidades_en_bloque = ner_pipeline(bloque_texto)
        entidades_agrupadas_bloque = agrupar_entidades_consecutivas(entidades_en_bloque)
        entidades.extend(entidades_agrupadas_bloque)

    return [entidad["word"] for entidad in entidades]

### Obtener embeddings

In [ ]:
def cargar_modelo(nombre_modelo):
    return SentenceTransformer(nombre_modelo)

# funcion para obtener los vectores de embeddings a partir de los textos y las entidades reconocidas
def obtener_embeddings(df, campo, modelo, ner_pipeline, max_tokens=512):
    tokenizador = modelo.tokenizer if hasattr(modelo, 'tokenizer') else None
    textos = df[campo].astype(str).tolist()
    embeddings = []

    for texto in textos:
        entidades = extraer_ners(texto, ner_pipeline)
        texto_enriquecido = f"{texto} {' '.join(entidades)}" # se crea un nuevo texto que combina el original con las entidades encontradas

        # se tokeniza el texto enriquecido con el tokenizador del modelo o se separa por espacios si no hay un tokenizador
        tokens = tokenizador.tokenize(texto_enriquecido) if tokenizador else texto_enriquecido.split()

        if len(tokens) <= max_tokens:
            embedding = modelo.encode(texto_enriquecido)
        else:
            embeddings_por_bloque = [] # si se excede el limite de tokens, se divide el texto en bloques
            
            for i in range(0, len(tokens), max_tokens):
                bloque_texto = " ".join(tokens[i:i+max_tokens])
                embedding_bloque = modelo.encode(bloque_texto)
                embeddings_por_bloque.append(embedding_bloque)
            embedding = sum(embeddings_por_bloque) / len(embeddings_por_bloque)

        embeddings.append(embedding)

    return embeddings

# funcion para guardar los embeddings generados en un archivo con formato binario
def guardar_embeddings(embeddings, archivo_salida):
    with open(archivo_salida, 'wb') as f:
        pickle.dump(embeddings, f)

In [3]:
protocolos=pd.read_csv("protocolos_completo_limpios.csv")

In [4]:
protocolos_acentuados=pd.read_csv("protocolos_completo_limpios.csv")

In [ ]:
def normalizar_texto(texto):
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto.lower()
protocolos = protocolos.applymap(lambda x: normalizar_texto(x) if isinstance(x, str) else x)

In [ ]:
def main():
    df = protocolos
    campo = input("Selecciona el campo (Titulo, resumen, objetivos, claves) para obtener sus PKL's: ").strip()

    if campo not in df.columns:
        print("El campo ingresado es invalido.")
        return

    ruta_destino = os.path.join("pkl")
    os.makedirs(ruta_destino, exist_ok=True)

    ner_pipeline = cargar_modelo_ner()

    for clave_modelo, nombre_modelo in MODELOS.items():
        print(f"\nProcesando modelo: {clave_modelo}")
        modelo = cargar_modelo(nombre_modelo)
        embeddings_obtenidos = obtener_embeddings(df, campo, modelo, ner_pipeline)
        archivo_embeddings = os.path.join(ruta_destino, f"{campo}_{clave_modelo}_embeddings.pkl")
        guardar_embeddings(embeddings_obtenidos, archivo_embeddings)

if __name__ == "__main__":
    main()

### Comparación de campos

In [ ]:
df = protocolos
df_resultados_acentuados  = protocolos_acentuados
ner_pipeline = cargar_modelo_ner()

while True:
    query = input("\nIngresa el texto para comparar (o escribe 'salir' para terminar): ").strip()
    if query.lower() == "salir":
        break

   
    print(f"\nConsulta original: {query}")
    entidades_query = extraer_ners(query, ner_pipeline)
    print("Entidades NER detectadas:", entidades_query)

    # si hay entidades, se enriquece un texto 
    query_enriquecida = f"{query} {' '.join(entidades_query)}" if entidades_query else query
    query_con_contexto = f"Este trabajo trata sobre {query_enriquecida}" # se agrega contexto para el modelo, pues si solo ponemos la entidad, el modelo no sabe de que se trata

    df_resultados_acentuados = protocolos_acentuados.copy()
    df_resultados_acentuados["sim_total"] = 0


    for clave_modelo, nombre_modelo in MODELOS.items():
        modelo = SentenceTransformer(nombre_modelo)
        # se genera el embedding de la consulta con contexto
        embedding_query = modelo.encode(query_con_contexto, convert_to_tensor=False)

        for campo in CAMPOS:
            # se construye el nombre del archivo .pkl que contiene los embeddings del campo
            nombre_pkl = f"{campo}_{clave_modelo}_embeddings.pkl"
            ruta_pkl = os.path.join("pkl", nombre_pkl)
            if not os.path.exists(ruta_pkl):
                print(f"Falta el archivo: {ruta_pkl}")
                continue

            with open(ruta_pkl, "rb") as f:
                embeddings_cargados = pickle.load(f)

            # se calcula la similitud coseno entre el embedding de la consulta y los del archivo
            simi_coseno = cosine_similarity([embedding_query], embeddings_cargados)[0]
            df_resultados_acentuados[f"{campo}_{clave_modelo}"] = simi_coseno
            # se acumula la similitud en la columna "sim_total" (ponderacion simple)
            df_resultados_acentuados["sim_total"] += simi_coseno

    df_resultados_acentuados = df_resultados_acentuados.sort_values("sim_total", ascending=False).head(5)
    print("\nTop 5 resultados más similares:")
    for i, row in df_resultados_acentuados.iterrows():
        print(f"\nTT: {row['TT']}")
        for campo in CAMPOS:
            resumen = str(row[campo])
            if len(resumen) > 180:
                resumen = resumen[:180] + "..."
            print(f"- {campo.capitalize()}: {resumen}")
        print(f"Similitud total: {row['sim_total']:.4f}")


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Consulta original: redes neuronales convolucionales
Entidades NER detectadas: ['redes neuronales convolucionales']

Top 5 resultados más similares:

TT: 2016-B026
- Titulo: Reconocimiento del Tránsito vehicular aplicando Redes Neuronales Profundas
- Resumen: Tomando en cuenta el incremento del congestionamiento vial y el aumento de cámaras en  las calles en la Ciudad de México proponemos desarrollar una herramienta que realice el  reco...
- Objetivos: Objetivos General Implementar una Red Neuronal Convolucional que haga uso del video tomado por cámaras  para reconocer y clasificar el congestionamiento vehicular. Particulares  Im...
- Claves: Deep Learning, Redes Neuronales Profundas o Convolucionales, Visión por Computadora.
Similitud total: 4.6636

TT: 2016-B029
- Titulo: Sistema de Clasificación de Razas Caninas Mediante Redes Neuronales Convolucionales
- Resumen: En el presente protocolo se describe una propuesta para desarrollar un sistema computacional con arquitectura cliente- s